In [ ]:
# See information about the GPUs you are using
# If you don't see GPUs, make sure you have enabled Kaggle's accelerators 
# Go to settings -> accelerator -> GPU T4x2
!nvidia-smi

# Download and install ollama to run within the Kaggle environment
!curl -fsSL https://ollama.com/install.sh | sh

# install necessary packages
! pip install -qq pyngrok ollama

# import Python dependencies
import subprocess
from pyngrok import ngrok
from kaggle_secrets import UserSecretsClient
import subprocess
import os

In [ ]:
# PARAMETERS
NGROK_PORT = '11434'

# auxiliary functions
def run(commands):
    for command in commands:
        with subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, 
                            stderr=subprocess.STDOUT, text=True, bufsize=1) as sp:
            for line in sp.stdout:
                if "undefined reference" in line:
                    raise RuntimeError("Failed Processing.")
                print(line, end="", flush=True)

def run(commands):
    for command in commands:
        with subprocess.Popen(command, shell = True, stdout = subprocess.PIPE, stderr = subprocess.STDOUT, bufsize = 1) as sp:
            for line in sp.stdout:
                line = line.decode("utf-8", errors = "replace")
                if "undefined reference" in line:
                    raise RuntimeError("Failed Processing.")
                print(line, flush = True, end = "")
        pass
    pass
pass

def start_ollama_server() -> None:
    """Starts the Ollama server."""
    # run(["ollama serve"])
    subprocess.Popen(['ollama', 'serve'])
    print("Ollama server started.")


def check_ollama_port(port: str) -> None:
    """Check if Ollama server is running at the specified port."""
    try:
        subprocess.run(['sudo', 'lsof', '-i', '-P', '-n'], check=True, capture_output=True, text=True)
        if any(f":{port} (LISTEN)" in line for line in subprocess.run(['sudo', 'lsof', '-i', '-P', '-n'], capture_output=True, text=True).stdout.splitlines()):
            print(f"Ollama is listening on port {port}")
        else:
            print(f"Ollama does not appear to be listening on port {port}.")
    except subprocess.CalledProcessError as e:
         print(f"Error checking Ollama port: {e}")


def setup_ngrok_tunnel(port: str, secret_manager: UserSecretsClient) -> ngrok.NgrokTunnel:
    """Sets up an ngrok tunnel.

    Args:
        port: The port to tunnel.

    Returns:
        The ngrok tunnel object.

    Raises:
        RuntimeError: If the ngrok authtoken is not set.
    """
    ngrok_auth_token = secret_manager.get_secret("NGROK_AUTHTOKEN")
    if not ngrok_auth_token:
        raise RuntimeError("NGROK_AUTHTOKEN is not set.")

    ngrok.set_auth_token(ngrok_auth_token)
    tunnel = ngrok.connect(port, host_header=f'localhost:{port}')
    print(f"ngrok tunnel created: {tunnel.public_url}")
    return tunnel

In [ ]:
# Start the ollama server and make sure it is running properly
start_ollama_server()
check_ollama_port(NGROK_PORT)

In [ ]:
# Setup ngrok tunnel to be able to access the ollama server from the outside world
user_secrets = UserSecretsClient()
ngrok_tunnel = setup_ngrok_tunnel(NGROK_PORT, user_secrets)

In [ ]:
# print(f'export OLLAMA_HOST={ngrok_tunnel.public_url}')

model = "gpt-oss"

commands = [
        f"ollama pull {model}",
        f"ollama run {model} \"create 10 sentences that ends with PAOK\""
        # f"curl {ngrok_tunnel.public_url}/api/chat -d '\{\"model\": \"{model}\", \"stream\": false, \"messages\": [\{ \"role\": \"user\", \"content\": \"create 10 sentences that ends with apple\" }]}'"
]
run(commands)